In [ ]:
library(tidyverse)
library(ggrepel)
library(ggthemes)
library(scales)
library(stringi)

In [ ]:
url <- "https://raw.githubusercontent.com/DieselAnalytics/PASS2018_PowerBI_GGPLOT2/master/Data/dataset.csv"
dataset <- read_csv(url)

In [ ]:
graph_data <-
    dataset %>%
    filter(`40YD` != 0 & BenchReps != 0) %>%
    mutate(
        Scaled.40YD = 
            round(rescale(`40YD` * -1, to = c(-10, 10)), 1)
        ,Scaled.BenchReps = 
            round(rescale(BenchReps, to = c(-10, 10)), 1)
    )
head(graph_data)

### Logic template for a custom R visual for Power BI


        if (< dataset test code >) {

            # <Put code to generate graph here>
    
        } else {
    
            p <- ggplot(dataset, aes(x = BenchReps, y = `40YD`)) +
                 geom_blank() +
                 scale_x_continuous(
                      labels = NULL
                    , breaks = NULL
                 ) +
                 scale_y_continuous(
                      labels = NULL
                    , breaks = NULL
                 ) +
                 labs(x = NULL, y = NULL) +
                 theme_classic()
            p
        }

In [ ]:
p <- ggplot(graph_data, aes(x = Scaled.BenchReps, y = Scaled.40YD, col = Position)) +
     geom_point() +
     geom_label_repel(aes(label = Player), size = 4, show.legend = FALSE) +
     geom_hline(yintercept = 0) +
     geom_vline(xintercept = 0) + 
     labs(
         title = "My Combine Quad Chart"
     )
p

In [ ]:
p <- p +
     xlab(bquote("Strength" ~ symbol('\256'))) +
     ylab(bquote("Speed" ~ symbol('\256')))
p

In [ ]:
p <- p +
     # quad labels
     annotate("text", x = -5, y = -11, label = "Average", alpha = 0.2, size = 6) +
     annotate("text", x = -5, y = 11, label = "Fast", alpha = 0.2, size = 6) +
     annotate("text", x = 5, y = -11, label = "Strong", alpha = 0.2, size = 6) +
     annotate("text", x = 5, y = 11, label = "Beast Mode", alpha = 0.2, size = 6)
p

In [ ]:
p <- p +
     annotate("rect", xmin = -3.5, xmax = -6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
     annotate("rect", xmin = -3.5, xmax = -6.5, ymin = 10.5, ymax = 11.5, alpha = .2) +
     annotate("rect", xmin = 3.5, xmax = 6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
     annotate("rect", xmin = 3.5, xmax = 6.5, ymin = 10.5, ymax = 11.5, alpha = .2)
p

In [ ]:
p <- p +
     theme_tufte()

In [ ]:
p <- p +
     annotate("rect", xmin = -Inf, xmax = 0.0, ymin = -Inf, ymax = 0, alpha = 0.1, fill = "lightskyblue") +
     annotate("rect", xmin = 0.0, xmax = Inf, ymin = 0.0, ymax = Inf, alpha = 0.1, fill = "lightskyblue")
p

In [ ]:
head(dataset)

In [ ]:
selected_position <- unique(dataset$`Position Group`)
chart.title <- paste(unique(dataset$Year), "Combine Quad Chart for", selected_position, sep = " ")
chart.title

In [ ]:
No40YDSubtitle <- 
    as.character(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]) %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not run the 40YD.") %>%
    str_wrap(width = 70, exdent = 2)
No40YDSubtitleHaveAthletes <-
    length(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]) %>%
    {ifelse(. > 0, "Y", "N")}

NoBenchPressSubtitle <- 
    as.character(dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)]) %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not participate in the benchpress.") %>%
    str_wrap(width = 70, exdent = 2)
NoBenchPressSubtitleHaveAthletes <- 
    length(dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)]) %>%
    {ifelse(. > 0, "Y", "N")}

No40YDAndBenchPressSubtitle <- 
    as.character(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)]) %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not participate in either the 40YD or in the benchpress.") %>%
    str_wrap(width = 70, exdent = 2)
No40YDAndBenchPressSubtitleHaveAthletes <- 
    length(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)]) %>%
    {ifelse(. > 0, "Y", "N")}


In [ ]:
No40YDSubtitle
NoBenchPressSubtitle
No40YDAndBenchPressSubtitle

In [ ]:
BestBench <- max(dataset$BenchReps)
BestBenchAthletes_cv <- as.character(dataset$Player[which(dataset$BenchReps == BestBench)])
BestBenchAthletesCount <- length(BestBenchAthletes_cv)
BestBenchAthletes <- 
    BestBenchAthletes_cv %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ")
BestBenchSubtitle <- 
    paste0(
        "- The highest number of reps for the benchpress was ",
        BestBench,
        ". ",
        BestBenchAthletes,
        ifelse(BestBenchAthletesCount > 1, " are ", " is "),
        "in this group"
    ) %>%
    str_wrap(width = 70, exdent = 2)

In [ ]:
    WorstBench <- min(dataset$BenchReps[dataset$BenchReps != 0])    
    WorstBenchAthletes_cv <- as.character(dataset$Player[which(dataset$BenchReps == WorstBench)])
    WorstBenchAthletesCount <- length(WorstBenchAthletes_cv)
    WorstBenchAthletes <- 
        WorstBenchAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    WorstBenchSubtitle <- 
        paste0(
            "- The lowest number of reps for the benchpress was ",
            WorstBench,
            ". ",
            WorstBenchAthletes,
            ifelse(WorstBenchAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 70, exdent = 2)


In [ ]:
    Best40YD <- min(dataset$`40YD`[dataset$`40YD` != 0])
    Best40YDAthletes_cv <- as.character(dataset$Player[which(dataset$`40YD` == Best40YD)])
    Best40YDAthletesCount <- length(Best40YDAthletes_cv)
    Best40YDAthletes <- 
        Best40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    Best40YDSubtitle <- 
        paste0(
            "- The fastest time for the 40YD was ",
            Best40YD,
            ". ",
            Best40YDAthletes,
            ifelse(Best40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 70, exdent = 2)

In [ ]:
    Worst40YD <- max(dataset$`40YD`)
    Worst40YDAthletes_cv <- as.character(dataset$Player[which(dataset$`40YD` == Worst40YD)])
    Worst40YDAthletesCount <- length(Worst40YDAthletes_cv)
    Worst40YDAthletes <- 
        Worst40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    Worst40YDSubtitle <- 
        paste0(
            "- The slowest time for the 40YD was ",
            Worst40YD,
            ". ",
            Worst40YDAthletes,
            ifelse(Worst40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 70, exdent = 2)

In [ ]:
    chartsubtitle <-
            paste0(
                ifelse(BestBenchAthletesCount == 0,"",paste0("\n", BestBenchSubtitle)),
                ifelse(WorstBenchAthletesCount == 0,"",paste0("\n", WorstBenchSubtitle)),
                ifelse(Best40YDAthletesCount == 0,"",paste0("\n", Best40YDSubtitle)),
                ifelse(Worst40YDAthletesCount == 0,"",paste0("\n", Worst40YDSubtitle)),
                ifelse(No40YDSubtitleHaveAthletes == "Y",paste0("\n", No40YDSubtitle),""),
                ifelse(NoBenchPressSubtitleHaveAthletes == "Y",paste0("\n", NoBenchPressSubtitle),""),
                ifelse(No40YDAndBenchPressSubtitleHaveAthletes == "Y",paste0("\n", No40YDAndBenchPressSubtitle),"")
            )

In [ ]:
    chart_source <- "Source:  https://www.pro-football-reference.com"

In [ ]:
p

In [ ]:
p <- p +
     labs(
         title = chart.title, 
         subtitle = chartsubtitle, 
         caption = chart_source
     )

In [ ]:
p <- p +
     theme(
          plot.title = element_text(hjust = 0.5, size = 25)
        , plot.subtitle = element_text(size = 15) 
        , panel.border = element_rect(colour = "black", size = 2, fill = NA)
        , axis.title.x = element_text(hjust = 0.1, size = 18)
        , axis.title.y = element_text(hjust = 0.1, size = 18)
     )
p

In [ ]:
library(tidyverse)
library(ggrepel)
library(ggthemes)
library(scales)
library(stringi)

if (length(unique(dataset$Year)) == 1 & length(unique(dataset$`Position Group`)) == 1) {
  
    selected_position <- unique(dataset$`Position Group`)
    chart.title <- paste(min(dataset$Year), "Combine Quad Chart for", selected_position, sep = " ")

    No40YDSubtitle <- 
        as.character(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]) %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ") %>%
        paste0("- ", .," did not run the 40YD.") %>%
        str_wrap(width = 140, exdent = 2)
    No40YDSubtitleHaveAthletes <-
        length(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]) %>%
        {ifelse(. > 0, "Y", "N")}
    
    NoBenchPressSubtitle <- 
        as.character(dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)]) %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ") %>%
        paste0("- ", .," did not participate in the benchpress.") %>%
        str_wrap(width = 140, exdent = 2)
    NoBenchPressSubtitleHaveAthletes <- 
        length(dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)]) %>%
        {ifelse(. > 0, "Y", "N")}
    
    No40YDAndBenchPressSubtitle <- 
        as.character(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)]) %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ") %>%
        paste0("- ", .," did not participate in either the 40YD or in the benchpress.") %>%
        str_wrap(width = 140, exdent = 2)
    No40YDAndBenchPressSubtitleHaveAthletes <- 
        length(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)]) %>%
        {ifelse(. > 0, "Y", "N")}

    graph_data <-
        dataset %>%
        filter(`40YD` != 0 & BenchReps != 0) %>%
        mutate(
            Scaled.40YD = 
                round(rescale(`40YD` * -1, to = c(-10, 10)), 1)
            ,Scaled.BenchReps = 
                round(rescale(BenchReps, to = c(-10, 10)), 1)
        )

    BestBench <- max(dataset$BenchReps)
    BestBenchAthletes_cv <- as.character(dataset$Player[which(dataset$BenchReps == BestBench)])
    BestBenchAthletesCount <- length(BestBenchAthletes_cv)
    BestBenchAthletes <- 
        BestBenchAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    BestBenchSubtitle <- 
        paste0(
            "- The highest number of reps for the benchpress was ",
            BestBench,
            ". ",
            BestBenchAthletes,
            ifelse(BestBenchAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)

    WorstBench <- min(dataset$BenchReps[dataset$BenchReps != 0])   
    WorstBenchAthletes_cv <- as.character(dataset$Player[which(dataset$BenchReps == WorstBench)])
    WorstBenchAthletesCount <- length(WorstBenchAthletes_cv)
    WorstBenchAthletes <- 
        WorstBenchAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    WorstBenchSubtitle <- 
        paste0(
            "- The lowest number of reps for the benchpress was ",
            WorstBench,
            ". ",
            WorstBenchAthletes,
            ifelse(WorstBenchAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)

    Best40YD <- min(dataset$`40YD`[dataset$`40YD` != 0])
    Best40YDAthletes_cv <- as.character(dataset$Player[which(dataset$`40YD` == Best40YD)])
    Best40YDAthletesCount <- length(Best40YDAthletes_cv)
    Best40YDAthletes <- 
        Best40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    Best40YDSubtitle <- 
        paste0(
            "- The fastest time for the 40YD was ",
            Best40YD,
            ". ",
            Best40YDAthletes,
            ifelse(Best40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)

    Worst40YD <- max(dataset$`40YD`)
    Worst40YDAthletes_cv <- as.character(dataset$Player[which(dataset$`40YD` == Worst40YD)])
    Worst40YDAthletesCount <- length(Worst40YDAthletes_cv)
    Worst40YDAthletes <- 
        Worst40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    Worst40YDSubtitle <- 
        paste0(
            "- The slowest time for the 40YD was ",
            Worst40YD,
            ". ",
            Worst40YDAthletes,
            ifelse(Worst40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)
    
    chartsubtitle <-
            paste0(
                ifelse(BestBenchAthletesCount == 0,"",paste0("\n", BestBenchSubtitle)),
                ifelse(WorstBenchAthletesCount == 0,"",paste0("\n", WorstBenchSubtitle)),
                ifelse(Best40YDAthletesCount == 0,"",paste0("\n", Best40YDSubtitle)),
                ifelse(Worst40YDAthletesCount == 0,"",paste0("\n", Worst40YDSubtitle)),
                ifelse(No40YDSubtitleHaveAthletes == "Y",paste0("\n", No40YDSubtitle),""),
                ifelse(NoBenchPressSubtitleHaveAthletes == "Y",paste0("\n", NoBenchPressSubtitle),""),
                ifelse(No40YDAndBenchPressSubtitleHaveAthletes == "Y",paste0("\n", No40YDAndBenchPressSubtitle),"")
            )
    
    chart_source <- "Source:  https://www.pro-football-reference.com"

    p <- ggplot(graph_data, aes(x = Scaled.BenchReps, y = Scaled.40YD, col = Position)) +
         geom_point() +
         geom_label_repel(aes(label = Player), size = 4, show.legend = FALSE) +
         geom_hline(yintercept = 0) +
         geom_vline(xintercept = 0) +

         # quad labels
         annotate("text", x = -5, y = -11, label = "Average", alpha = 0.2, size = 6) +
         annotate("text", x = -5, y = 11, label = "Fast", alpha = 0.2, size = 6) +
         annotate("text", x = 5, y = -11, label = "Strong", alpha = 0.2, size = 6) +
         annotate("text", x = 5, y = 11, label = "Beast Mode", alpha = 0.2, size = 6) +

         # Squares for quad labels
         annotate("rect", xmin = -3.5, xmax = -6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
         annotate("rect", xmin = -3.5, xmax = -6.5, ymin = 10.5, ymax = 11.5, alpha = .2) +
         annotate("rect", xmin = 3.5, xmax = 6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
         annotate("rect", xmin = 3.5, xmax = 6.5, ymin = 10.5, ymax = 11.5, alpha = .2) +

         # Shade lower left quadrant
         annotate("rect", xmin = -Inf, xmax = 0.0, ymin = -Inf, ymax = 0, alpha = 0.1, fill = "lightskyblue") +
        
         # Shade upper right quadrant
         annotate("rect", xmin = 0.0, xmax = Inf, ymin = 0.0, ymax = Inf, alpha = 0.1, fill = "lightskyblue") +
        
         # Titles
         xlab(bquote("Strength" ~ symbol('\256'))) +
         ylab(bquote("Speed" ~ symbol('\256'))) +
         #ggtitle(chart.title, subtitle = chartsubtitle) +
         labs(title = chart.title, subtitle = chartsubtitle, caption = chart_source) +

         # Prettying things up
         theme_tufte() +
         theme(
              plot.title = element_text(hjust = 0.5, size = 25)
            , plot.subtitle = element_text(size = 15) 
            , panel.border = element_rect(colour = "black", size = 2, fill = NA)
            , axis.title.x = element_text(hjust = 0.1, size = 18)
            , axis.title.y = element_text(hjust = 0.1, size = 18)
         ) +
         scale_x_continuous(labels = NULL, breaks = NULL) +
         scale_y_continuous(labels = NULL, breaks = NULL) 
    
         p

} else {
    p <- ggplot(dataset, aes(x = BenchReps, y = `40YD`)) +
         geom_blank() +
         scale_x_continuous(
              labels = NULL
            , breaks = NULL
        ) +
         scale_y_continuous(
              labels = NULL
            , breaks = NULL
         ) +
         labs(x = NULL, y = NULL) +
         theme_classic()
    p
}